<a href="https://colab.research.google.com/github/BFvandy/Nissan-Text-Mining-POC-with-Transformer/blob/main/SW_Zeroshot_after_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
cur_path = "/content/drive/My Drive/Nissan Internship/SW 2020-2022 Verbatim Small SUV"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/Nissan Internship/SW 2020-2022 Verbatim Small SUV


# Load Transformer

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 54.9 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

# Load Data

In [ ]:
comments = pd.read_csv('SW_cmt_prefiltered.csv', encoding_errors='ignore')

In [ ]:
test_df = comments.copy()

In [ ]:
pd.set_option('display.max_columns', None)
comments.head(10)

,Make,Model,Purchased,Study Year,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,Age Range,Gender,Ethnicity,Income Bracket,Text,dependable,lasts long,value for money,quality fit and finish,attractive styling,safe,retains resale value,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable,trusted,leader,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Honda,CR-V,Honda CR-V Special Edition FWD,2022,2021,Honda Civic,2019.0,Honda HR-V,2022.0,Sentiment Not Calculated,Age 41,Female,White,Income Unknown,The engine start stop/ gas save mode is not ve...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Nissan,Rogue,Nissan Rogue SV AWD,2022,2021,Ford Fusion,2012.0,Toyota Highlander,2021.0,Sentiment Not Calculated,Age 35,Male,White,"$100,001 - $125,000",We were not told the price of the vehicle goin...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Nissan,Rogue,Nissan Rogue S AWD,2022,2021,Chevrolet Spark,2016.0,Hyundai Santa Fe,2021.0,Sentiment Not Calculated,Age 57,Male,White,Income Unknown,My Nissan Rogue is the perfect balance between...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,Nissan Versa Note (Hatchback),2018.0,Lexus LX,2022.0,Sentiment Not Calculated,Age 49,Female,Asian,"$65,001 - $70,000",I wish :\n1.there is a CD player installed. T...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,(None),NaN,Honda Civic,2015.0,Sentiment Not Calculated,Age 54,Female,White,"$200,001 - $300,000","The 2022 Honda CRV is a nice looking car, with...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Honda,CR-V,Honda CR-V Touring AWD,2022,2022,(None),NaN,Ford Edge,2022.0,Sentiment Not Calculated,Age 64,Female,White,Income Unknown,Here’s my suggestions to improve this vehicle:...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Honda,CR-V,Honda CR-V Touring Hybrid AWD,2022,2022,Honda CR-V,2006.0,Toyota RAV4,2022.0,Sentiment Not Calculated,Age 63,Female,White,"$50,001 - $55,000",This is not the CRV of the past! It does not h...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Honda,CR-V,Honda CR-V EX/EX-L AWD,2022,2021,Honda HR-V,2019.0,(None),NaN,Sentiment Not Calculated,Age 65,Female,White,"$35,001 - $40,000",i was very disappointed in your discrimatory a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Honda,CR-V,Honda CR-V Touring AWD,2022,2021,Toyota RAV4,2018.0,Toyota RAV4,2022.0,Sentiment Not Calculated,Age 39,Female,White,"$150,001 - $175,000",The engine noise and rattling while accelerati...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,Honda,CR-V,Honda CR-V Touring Hybrid AWD,2022,2022,Honda CR-V,2009.0,Volvo XC40,2022.0,Sentiment Not Calculated,Age Unknown,Female,Asian,Income Unknown,It was a very bad experience purchasing the ca...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
comments.shape

(107004, 44)

In [ ]:
comments.dtypes

Make                         object
Model                        object
Purchased                    object
Study Year                    int64
Purchased Year                int64
 Disposed                    object
Disposed Model Year         float64
Rejected                     object
Rejected Model Year         float64
Sentiment                    object
Age Range                    object
Gender                       object
Ethnicity                    object
Income Bracket               object
Text                         object
dependable                  float64
lasts long                  float64
value for money             float64
quality fit and finish      float64
attractive styling          float64
safe                        float64
retains resale value        float64
driver comfort              float64
fun to drive                float64
advanced features           float64
responsive handling         float64
prestigious                 float64
dealerships                 

In [ ]:
#all the labels
# 17 functional attributes
dependable_labels = ['dependable', 'not dependable']
lasts_labels = ['lasts long', 'not lasts long']
value_labels = ['value for money', 'not value for money']
quality_labels = ['quality fit and finish', 'no quality fit and finish']
attractive_labels = ['attractive styling', 'not attractive styling']
safe_labels = ['safe', 'not safe']
retains_labels	= ['retains resale value', 'not retains resale value']
driver_labels	= ['driver comfort', 'driver not comfortable']
fun_labels = ['fun to drive', 'not fun to drive']
advanced_labels = ['advanced features', 'no advanced features']
responsive_labels = ['responsive handling', 'not responsive handling']
prestigious_labels = ['prestigious', 'not prestigious']
dealerships_labels = ['good dealerships', 'bad dealerships']
fuel_labels = ['fuel efficient', 'not fuel efficient']
quick_labels = ['quick acceleration', 'slow acceleration']
environ_labels = ['environmentally friendly', 'not environmentally friendly']
affordable_labels = ['affordable', 'not affordable']

# 12 personality attributes
trust_label = ['trusted', 'not trusted']
leader_label = ['leader', 'not leader']
resp_label = ['responsible', 'not responsible']
conf_label = ['confident','not confident']
innov_label = ['innovative', 'not innovative'] 
excit_label = ['exciting', 'not exciting']
pract_label = ['practical', 'not practical']
adven_label = ['adventurous', 'not adventurous']
pass_label = ['passionate', 'not passionate']
dist_label = ['distinctive','not distinctive']
youth_label = ['youthful', 'not youthful']
aggre_label = ['aggressive', 'not aggressive']


In [ ]:
funct_labels = [dependable_labels, lasts_labels, value_labels, quality_labels, attractive_labels, safe_labels, retains_labels, driver_labels, fun_labels, advanced_labels, responsive_labels, prestigious_labels, dealerships_labels, fuel_labels, quick_labels, environ_labels, affordable_labels]
pers_labels = [trust_label, leader_label, resp_label, conf_label, innov_label, excit_label, pract_label, adven_label, pass_label, dist_label, youth_label, aggre_label]

In [ ]:
all_labels = funct_labels + pers_labels
all_labels

[['dependable', 'not dependable'],
 ['lasts long', 'not lasts long'],
 ['value for money', 'not value for money'],
 ['quality fit and finish', 'no quality fit and finish'],
 ['attractive styling', 'not attractive styling'],
 ['safe', 'not safe'],
 ['retains resale value', 'not retains resale value'],
 ['driver comfort', 'driver not comfortable'],
 ['fun to drive', 'not fun to drive'],
 ['advanced features', 'no advanced features'],
 ['responsive handling', 'not responsive handling'],
 ['prestigious', 'not prestigious'],
 ['good dealerships', 'bad dealerships'],
 ['fuel efficient', 'not fuel efficient'],
 ['quick acceleration', 'slow acceleration'],
 ['environmentally friendly', 'not environmentally friendly'],
 ['affordable', 'not affordable'],
 ['trusted', 'not trusted'],
 ['leader', 'not leader'],
 ['responsible', 'not responsible'],
 ['confident', 'not confident'],
 ['innovative', 'not innovative'],
 ['exciting', 'not exciting'],
 ['practical', 'not practical'],
 ['adventurous', 'no

In [ ]:
len(all_labels)

29

# Implement Zero-shot Model

In [ ]:
for i in range(len(comments)):
  for j in range(29):
    #29 labels in total
    column_num = j + 15
    #match column index with label index
    if comments.iat[i, column_num] != 0:
      try:
        res = classifier(comments.at[i, 'Text'], all_labels[j], multi_label=False)
        ind = res['labels'].index(all_labels[j][0])
        comments.iat[i, column_num] = res.get('scores')[ind]
      except:
        pass
  if i % 5000 == 0:
    print(i)
  else:
    pass


0


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [ ]:
comments.to_csv('SW_score_v1.csv',index = False)

In [ ]:
comments

,Make,Model,Purchased,Study Year,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,Age Range,Gender,Ethnicity,Income Bracket,Text,dependable,lasts long,value for money,quality fit and finish,attractive styling,safe,retains resale value,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable,trusted,leader,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Honda,CR-V,Honda CR-V Special Edition FWD,2022,2021,Honda Civic,2019.0,Honda HR-V,2022.0,Sentiment Not Calculated,Age 41,Female,White,Income Unknown,The engine start stop/ gas save mode is not ve...,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Nissan,Rogue,Nissan Rogue SV AWD,2022,2021,Ford Fusion,2012.0,Toyota Highlander,2021.0,Sentiment Not Calculated,Age 35,Male,White,"$100,001 - $125,000",We were not told the price of the vehicle goin...,0.0,0.000000,0.029433,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.011031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Nissan,Rogue,Nissan Rogue S AWD,2022,2021,Chevrolet Spark,2016.0,Hyundai Santa Fe,2021.0,Sentiment Not Calculated,Age 57,Male,White,Income Unknown,My Nissan Rogue is the perfect balance between...,0.0,0.000000,0.000000,0.984863,0.98799,0.000000,0.0,0.0,0.997018,0.0,0.993778,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.974088,0.0,0.0,0.0,0.0,0.0,0.0
3,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,Nissan Versa Note (Hatchback),2018.0,Lexus LX,2022.0,Sentiment Not Calculated,Age 49,Female,Asian,"$65,001 - $70,000",I wish :\n1.there is a CD player installed. T...,0.0,0.640963,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,(None),NaN,Honda Civic,2015.0,Sentiment Not Calculated,Age 54,Female,White,"$200,001 - $300,000","The 2022 Honda CRV is a nice looking car, with...",0.0,0.000000,0.000000,0.000000,0.00000,0.048322,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106999,Honda,CR-V,Honda CR-V EX/EX-L FWD,2020,2020,None,18.0,None,16.0,Sentiment Not Calculated,Age 34,Female,Hispanic,"$150,001 - $175,000",G,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
107000,Honda,CR-V,Honda CR-V EX/EX-L AWD,2020,2019,GMC Terrain,2013.0,Honda CR-V,2020.0,Sentiment Not Calculated,Age 42,Male,White,"$100,001 - $125,000",.,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
107001,Honda,CR-V,Honda CR-V EX/EX-L AWD,2020,2019,GMC Terrain,2013.0,Honda CR-V,2020.0,Sentiment Not Calculated,Age 42,Male,White,"$100,001 - $125,000",.,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
107002,Honda,CR-V,Honda CR-V LX FWD,2020,2019,Honda CR-V,2015.0,Honda HR-V,2019.0,Sentiment Not Calculated,Age 59,Male,White,"$50,001 - $55,000",I,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
